In [1]:
#imports
import numpy as np
import pandas as pd

import time
import requests

In [2]:
#In order to get the required 1000 posts, I would need to run API 10 times, while using the first 
#(chronologically) post date as before parameter
#Lets run this and if it doesn't work, I'll add some timeout between the cycles. 
#Also lets run it 11 times so that we have some wiggle room 
url='https://api.pushshift.io/reddit/search/submission'

In [3]:
#testing
params={
    'subreddit':'AmItheAsshole',
    'size':100,
}
res=requests.get(url,params)
data=res.json()
posts=data['data']
df=pd.DataFrame(posts)
df[['subreddit','selftext','title','created_utc']]
#ok, it works. I plan to only use these four columns. Lets go ahead wit the loop. 

,subreddit,selftext,title,created_utc
0,AmItheAsshole,[removed],WIBTA if I don't come due to a family gathering?,1666933043
1,AmItheAsshole,So me (22F) and my boyfriend (25M) had a kid a...,AITA for yelling at him?,1666932981
2,AmItheAsshole,[removed],AITA for not making Divorce an Easy option for...,1666932878
3,AmItheAsshole,[removed],AITA for not asking my friend out first?,1666932375
4,AmItheAsshole,[removed],AITA for getting mad at my husband for playing...,1666932288
...,...,...,...,...
95,AmItheAsshole,[removed],AITA for getting mad at my mom for not using m...,1666925476
96,AmItheAsshole,[removed],AITA for speaking up about another member in m...,1666925441
97,AmItheAsshole,I (23F) have 3 roommates A(21F) D (22F) and W\...,AITA for intentionally throwing my roommates' ...,1666925409
98,AmItheAsshole,I (21f) am in college and have been finding it...,AITA for talking too much,1666925399


In [4]:
#now that I think of it, since posts are getting erased all the time, I migh need more than 11 cycles
#it complains about too many requests. 
df_aita=pd.DataFrame()
#this code is usable, but there is no needo
for i in range(10):
    res=requests.get(url,params)
    data=res.json()
    posts=data['data']
    df=pd.DataFrame(posts)
    df_aita=pd.concat([df[['subreddit','selftext','title']],df_aita])
    post_date=df['created_utc'].tolist()[-1]
    params={
    'subreddit':'AmItheAsshole',
    'size':100,
    'before':post_date
    }
    time.sleep(2)
df_aita.shape

(999, 3)

In [5]:
#I think it went fine. Lets see how many posts were removed, then adjust our sample size
df_aita[df_aita['selftext']=='[removed]']
#yikes, its 1/3. Lets take a sample of 2100 and then remove ever [removed] post. 

,subreddit,selftext,title
2,AmItheAsshole,[removed],AITA for getting mad at my daughter because sh...
5,AmItheAsshole,[removed],AITA for getting mad at my friend's for bailin...
6,AmItheAsshole,[removed],AITA for not raising my concerns about my Cous...
7,AmItheAsshole,[removed],AITA for devouring my friend's corpse and shov...
8,AmItheAsshole,[removed],AITA for being making them wait (with reason) ...
...,...,...,...
84,AmItheAsshole,[removed],WIBTA if I dated my brothers crush?
91,AmItheAsshole,[removed],AITA for pushing my disabled aunt down the sta...
94,AmItheAsshole,[removed],AITA for contacting an exdate after potentiall...
95,AmItheAsshole,[removed],AITA for getting mad at my mom for not using m...


In [6]:
df_aita=pd.DataFrame()
for i in range(21):
    res=requests.get(url,params)
    data=res.json()
    posts=data['data']
    df=pd.DataFrame(posts)
    df_aita=pd.concat([df[['subreddit','selftext','title']],df_aita])
    post_date=df['created_utc'].tolist()[-1]
    params={
    'subreddit':'AmItheAsshole',
    'size':100,
    'before':post_date
    }
    time.sleep(2)
df_aita.shape

(2098, 3)

In [7]:
df_aita=df_aita[df_aita['selftext']!='[removed]']
df_aita

,subreddit,selftext,title
0,AmItheAsshole,I had my junior prom. I was really excited for...,AITA for disagreeing with my parents?
2,AmItheAsshole,Throwaway and on mobile so please ignore bad f...,AITA for naming my child something similar to ...
3,AmItheAsshole,(Sorry for phone formatting) \n\nI (22m) was b...,WIBTA if I stayed quiet about what my niece said?
4,AmItheAsshole,So me and Z have been friends since last year ...,WIBTAH if i gave the cold shoulder to a friend...
5,AmItheAsshole,My wife has seen those families on tiktok that...,AITA for saying no to my wife’s idea of living...
...,...,...,...
92,AmItheAsshole,I (14f) do paper rounds in my small town. Toda...,AITA for moving someone’s bins without their p...
93,AmItheAsshole,Sorry for the bad formatting I'm on mobile.\n\...,AITA For Being Distrustful Of My Girlfriend?
95,AmItheAsshole,Hi everybody. I’m posting this through my daug...,AITA for canceling a dinner party during it?
96,AmItheAsshole,I (22f) and my friend (23f) have known each ot...,AITA for telling my friend to break up with he...


In [8]:
#the rate is consistent. This looks good. Now lets do the same thing and get 1600 posts for the second subreddit.
df_rela=pd.DataFrame()
params={
    'subreddit':'relationships',
    'size':100,
}
for i in range(16):
    res=requests.get(url,params)
    data=res.json()
    posts=data['data']
    df=pd.DataFrame(posts)
    df_rela=pd.concat([df[['subreddit','selftext','title']],df_rela])
    post_date=df['created_utc'].tolist()[-1]
    params={
    'subreddit':'relationships',
    'size':100,
    'before':post_date
    }
    time.sleep(2)
df_rela

,subreddit,selftext,title
0,relationships,tl;dr\n\nI am 18M dating a 18F(both almost 19)...,What should I do?
1,relationships,I (28F) have just been proposed to my boyfrien...,I asked my boyfriend to redo his proposal to m...
2,relationships,\n\nmy boyfriend has been having some self con...,How do I (F23) make my partner (M26) feel attr...
3,relationships,Broke up three days ago. My ex GF and I had be...,I (30M) just broke up with my GF (29F) and am ...
4,relationships,[removed],What should i do?
...,...,...,...
95,relationships,[removed],Counselor Issues 35 M
96,relationships,Ok so I have serious relationship anxiety and ...,My (24) bf (26) said he needs space does he re...
97,relationships,[removed],Counselor Issues
98,relationships,my gf had no self awareness and it's making me...,my gf cant and wont say no


In [9]:
#you are kidding me. 60% are getting removed. I need 3500 posts to match the samples from aita.
#I regret everything. 
df_rela[df_rela['selftext']=='[removed]']

,subreddit,selftext,title
4,relationships,[removed],What should i do?
5,relationships,[removed],How do I (F23) make my partner (M26) feel attr...
6,relationships,[removed],"I'm now engaged, but I still dream about my hi..."
7,relationships,[removed],My Girlfriend of 3+ Years Suddenly in a Long D...
9,relationships,[removed],My moms trying to make my wedding a family reu...
...,...,...,...
93,relationships,[removed],I think this girl likes me but I can’t tell
94,relationships,[removed],What are your pick-me-up’s for your SO?
95,relationships,[removed],Counselor Issues 35 M
97,relationships,[removed],Counselor Issues


In [10]:
df_rela=pd.DataFrame()
params={
    'subreddit':'relationships',
    'size':100,
}
for i in range(35):
    res=requests.get(url,params)
    data=res.json()
    posts=data['data']
    df=pd.DataFrame(posts)
    df_rela=pd.concat([df[['subreddit','selftext','title']],df_rela])
    post_date=df['created_utc'].tolist()[-1]
    params={
    'subreddit':'relationships',
    'size':100,
    'before':post_date
    }
    time.sleep(2)
df_rela=df_rela[df_rela['selftext']!='[removed]']
df_rela.shape

(1442, 3)

In [11]:
#2 second delay really didn't seem like a lot on 10-cycle loop. 
dff=pd.concat([df_rela,df_aita])

dff.to_csv('data/reddit.csv',index=False)
#I think this concludes the acquisition of the data. Lets see if  we can do some EDA. 